In [1]:
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import *
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
smartcities100 = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/smartcities100/smartcities100 tweets.csv')

In [3]:
smartcities100.head()

,tweetDate,content,twitterProfile,tweetUrl,timestamp,query,rank2020
0,Sun Jan 17 10:57:51 +0000 2021,A Delegation from @aau_ae visited @BurjeelMed...,https://twitter.com/Atatreh,https://twitter.com/Atatreh/status/13507592313...,2021-01-21T13:59:48.147Z,#AbuDhabi,42
1,Thu Jan 14 17:40:03 +0000 2021,Wizz Air #AbuDhabi is set to launch flights to...,https://twitter.com/UAE_Forsan,https://twitter.com/UAE_Forsan/status/13497732...,2021-01-21T13:59:48.147Z,#AbuDhabi,42
2,Wed Jan 20 20:25:02 +0000 2021,What a great grappling exchange by both man #A...,https://twitter.com/RdosAnjosMMA,https://twitter.com/RdosAnjosMMA/status/135198...,2021-01-21T13:59:48.147Z,#AbuDhabi,42
3,Sun Jan 17 16:55:17 +0000 2021,Totally worth the 3.5 hour drive to #AbuDhabi ...,https://twitter.com/zoomnclick,https://twitter.com/zoomnclick/status/13508491...,2021-01-21T13:59:48.147Z,#AbuDhabi,42
4,Mon Jan 18 04:44:44 +0000 2021,Good morning #AbuDhabi Ireland 🇮🇪 vs UAE 🇦🇪 fi...,https://twitter.com/ChTahirmehmood,https://twitter.com/ChTahirmehmood/status/1351...,2021-01-21T13:59:48.147Z,#AbuDhabi,42


In [4]:
# how many smartcities are there in my dataset ?
smartcities100['query'].nunique()

106

I will append my 3 case studies Taipei, Tallinn, Tel Aviv which were stored in dedicated files.

In [5]:
taipei = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Taipei/#Taipei.csv')
telaviv = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tel Aviv/#Telaviv.csv')
tallinn = pd.read_csv('/Users/juliencarbonnell/Desktop/Thèse/DONNÉES/1.Twitter/Tallinn/#Tallinn.csv')

The ranking2020 column is missing on these 3 files. Will add it before merging the files in the same dataframe

In [6]:
taipei['rank2020'] = '8'
telaviv['rank2020'] = '50'
tallinn['rank2020'] = '59'

In [7]:
smartcities100 = pd.concat([smartcities100, taipei,telaviv,tallinn], axis=0)

In [8]:
smartcities100.shape

(110862, 7)

In [9]:
smartcities100['query'].nunique()

109

In [10]:
smartcities100['query'].value_counts()

#telaviv       17612
#Taipei        17337
#Tallinn        6295
#Brisbane       2080
#Manchester     1877
               ...  
#Hanover         117
#Makassar        112
#Gothenburg       84
#Medan            71
#Busan            64
Name: query, Length: 109, dtype: int64

In [11]:
smartcities100.head()

,tweetDate,content,twitterProfile,tweetUrl,timestamp,query,rank2020
0,Sun Jan 17 10:57:51 +0000 2021,A Delegation from @aau_ae visited @BurjeelMed...,https://twitter.com/Atatreh,https://twitter.com/Atatreh/status/13507592313...,2021-01-21T13:59:48.147Z,#AbuDhabi,42
1,Thu Jan 14 17:40:03 +0000 2021,Wizz Air #AbuDhabi is set to launch flights to...,https://twitter.com/UAE_Forsan,https://twitter.com/UAE_Forsan/status/13497732...,2021-01-21T13:59:48.147Z,#AbuDhabi,42
2,Wed Jan 20 20:25:02 +0000 2021,What a great grappling exchange by both man #A...,https://twitter.com/RdosAnjosMMA,https://twitter.com/RdosAnjosMMA/status/135198...,2021-01-21T13:59:48.147Z,#AbuDhabi,42
3,Sun Jan 17 16:55:17 +0000 2021,Totally worth the 3.5 hour drive to #AbuDhabi ...,https://twitter.com/zoomnclick,https://twitter.com/zoomnclick/status/13508491...,2021-01-21T13:59:48.147Z,#AbuDhabi,42
4,Mon Jan 18 04:44:44 +0000 2021,Good morning #AbuDhabi Ireland 🇮🇪 vs UAE 🇦🇪 fi...,https://twitter.com/ChTahirmehmood,https://twitter.com/ChTahirmehmood/status/1351...,2021-01-21T13:59:48.147Z,#AbuDhabi,42


# 1. extract some tweet-level features
- tweet length for each tweet
- number of words for each tweet
- number of sentences for each tweet
- average word length for each tweet

In [12]:
# how many null values in content column ?
smartcities100['content'].isna().sum()

12

In [13]:
# keep only rows when content column value is not null
smartcities100 = smartcities100[smartcities100['content'].notnull()]

In [14]:
smartcities100['content'].isna().sum()

0

In [15]:
# add a column with tweet length for each tweet
smartcities100['tweet_len'] = smartcities100['content'].apply(lambda x: len(x))

In [16]:
#add a column with number of words for each tweet
smartcities100['num_words'] = smartcities100['content'].apply(lambda x: len(x.split()))

In [17]:
#add a column with number of sentences for each tweet
smartcities100['num_sentences'] = smartcities100['content'].apply(lambda x: len(re.split( '~ ...' ,'~'.join(x.split('.')))))

In [18]:
# create function for finding average length of words per tweet
def get_avg_word_len(x):
    words = x.split()
    word_len = 0
    for word in words:
        word_len = word_len + len(word)
        
    return word_len / len(words)

# add a column with average word length for each tweet
smartcities100['avg_word_len'] = smartcities100['content'].apply(lambda x: get_avg_word_len(x))

In [19]:
# use vader sentiment to get % of pos, neg, neu in new columns
analyzer= SentimentIntensityAnalyzer()

smartcities100['neg'] = [analyzer.polarity_scores(v)['neg'] for v in smartcities100['content']]
smartcities100['neu'] = [analyzer.polarity_scores(v)['neu'] for v in smartcities100['content']]
smartcities100['pos'] = [analyzer.polarity_scores(v)['pos'] for v in smartcities100['content']]
smartcities100['compound'] = [analyzer.polarity_scores(v)['compound'] for v in smartcities100['content']]

In [20]:
#check the columns of my dataset
smartcities100.head()

,tweetDate,content,twitterProfile,tweetUrl,timestamp,query,rank2020,tweet_len,num_words,num_sentences,avg_word_len,neg,neu,pos,compound
0,Sun Jan 17 10:57:51 +0000 2021,A Delegation from @aau_ae visited @BurjeelMed...,https://twitter.com/Atatreh,https://twitter.com/Atatreh/status/13507592313...,2021-01-21T13:59:48.147Z,#AbuDhabi,42,303,38,2,6.973684,0.104,0.849,0.047,-0.5267
1,Thu Jan 14 17:40:03 +0000 2021,Wizz Air #AbuDhabi is set to launch flights to...,https://twitter.com/UAE_Forsan,https://twitter.com/UAE_Forsan/status/13497732...,2021-01-21T13:59:48.147Z,#AbuDhabi,42,87,12,1,6.333333,0.000,1.000,0.000,0.0000
2,Wed Jan 20 20:25:02 +0000 2021,What a great grappling exchange by both man #A...,https://twitter.com/RdosAnjosMMA,https://twitter.com/RdosAnjosMMA/status/135198...,2021-01-21T13:59:48.147Z,#AbuDhabi,42,53,9,1,5.000000,0.000,0.661,0.339,0.6249
3,Sun Jan 17 16:55:17 +0000 2021,Totally worth the 3.5 hour drive to #AbuDhabi ...,https://twitter.com/zoomnclick,https://twitter.com/zoomnclick/status/13508491...,2021-01-21T13:59:48.147Z,#AbuDhabi,42,114,18,1,5.388889,0.000,0.748,0.252,0.6590
4,Mon Jan 18 04:44:44 +0000 2021,Good morning #AbuDhabi Ireland 🇮🇪 vs UAE 🇦🇪 fi...,https://twitter.com/ChTahirmehmood,https://twitter.com/ChTahirmehmood/status/1351...,2021-01-21T13:59:48.147Z,#AbuDhabi,42,91,12,1,6.666667,0.000,0.791,0.209,0.4404


In [21]:
# Save new .csv file with lastly created columns
smartcities100.to_csv('smartcities100_.csv', index=False)

Now my dataset is complete and we can see that my 3 case studies are bigger samples from the other cities. This unbalance is not annoying since I will work on means and standard deviations values. It gives my samples more proof when compared to the whole population. 

# 2. Extract city-level features 
- create a dataframe with city_id as index
- use groupby on the 'query' column 
- aggregate city-level features from the tweet-level dataframe 
- take the weight of smartcityBoWs for each city

## Create a new dataframe to store features 

In [22]:
# copy the unique values from 'query' column as index in the new dataframe
index = smartcities100['query'].unique()
columns = []

# create a new DataFrame
sc100_features = pd.DataFrame(index=index, columns=columns)

In [23]:
# add rank2020 column
sc100_features = pd.concat([sc100_features, smartcities100.groupby('query').rank2020.agg(['min'])], axis=1)
sc100_features

,min
#AbuDhabi,42
#Abuja,107
#Amsterdam,9
#Ankara,57
#Athens,99
...,...
#Zurich,3
#Taipei,8
NaN,NaN
#telaviv,50


In [24]:
# agg num_tweets column
sc100_features = pd.concat([sc100_features, smartcities100['query'].value_counts()], axis=1)
sc100_features

,min,query
#AbuDhabi,42,832.0
#Abuja,107,828.0
#Amsterdam,9,1000.0
#Ankara,57,313.0
#Athens,99,1000.0
...,...,...
#Zurich,3,485.0
#Taipei,8,17337.0
NaN,NaN,NaN
#telaviv,50,17612.0


In [25]:
# agg total tweet length by city
sc100_features['total_tweet_len'] = smartcities100.groupby('query')['tweet_len'].sum()
sc100_features

,min,query,total_tweet_len
#AbuDhabi,42,832.0,161350.0
#Abuja,107,828.0,173587.0
#Amsterdam,9,1000.0,180311.0
#Ankara,57,313.0,58158.0
#Athens,99,1000.0,186752.0
...,...,...,...
#Zurich,3,485.0,94012.0
#Taipei,8,17337.0,2505360.0
NaN,NaN,NaN,NaN
#telaviv,50,17612.0,2468640.0


In [26]:
# calculate average tweet length
sc100_features['avg_tweet_len'] = sc100_features['total_tweet_len']/sc100_features['query']

In [27]:
# agg total number of words 
sc100_features['total_num_words'] = smartcities100.groupby('query')['num_words'].sum()

In [28]:
# calculate average number of words
sc100_features['avg_num_words'] = sc100_features['total_num_words']/sc100_features['query']

In [29]:
# agg total number of sentences 
sc100_features['total_num_sentences'] = smartcities100.groupby('query')['num_sentences'].sum()

In [30]:
# calculate average number of sentences
sc100_features['avg_num_sentences'] = sc100_features['total_num_sentences']/sc100_features['query']

In [31]:
# agg total average word length
sc100_features['total_avg_word_len'] = smartcities100.groupby('query')['avg_word_len'].sum()

In [32]:
# calculate average of average word length
sc100_features['avg_avg_word_len'] = sc100_features['total_avg_word_len']/sc100_features['query']

In [33]:
# agg total sentiment scores
sc100_features['neg'] = smartcities100.groupby('query')['neg'].sum()
sc100_features['neu'] = smartcities100.groupby('query')['neu'].sum()
sc100_features['pos'] = smartcities100.groupby('query')['pos'].sum()
sc100_features['compound'] = smartcities100.groupby('query')['compound'].sum()

In [34]:
# calculate average sentiment scores
sc100_features['avg_neg'] = sc100_features['neg']/sc100_features['query']
sc100_features['avg_neu'] = sc100_features['neu']/sc100_features['query']
sc100_features['avg_pos'] = sc100_features['pos']/sc100_features['query']
sc100_features['avg_compound'] = sc100_features['compound']/sc100_features['query']

In [35]:
sc100_features

,min,query,total_tweet_len,avg_tweet_len,total_num_words,avg_num_words,total_num_sentences,avg_num_sentences,total_avg_word_len,avg_avg_word_len,neg,neu,pos,compound,avg_neg,avg_neu,avg_pos,avg_compound
#AbuDhabi,42,832.0,161350.0,193.930288,20802.0,25.002404,1194.0,1.435096,5887.253316,7.076026,18.248,715.630,98.123,249.5025,0.021933,0.860132,0.117936,0.299883
#Abuja,107,828.0,173587.0,209.646135,22391.0,27.042271,1190.0,1.437198,5858.506206,7.075491,15.064,709.099,103.828,295.2426,0.018193,0.856400,0.125396,0.356573
#Amsterdam,9,1000.0,180311.0,180.311000,22627.0,22.627000,1520.0,1.520000,7579.218712,7.579219,24.567,875.009,100.439,250.7977,0.024567,0.875009,0.100439,0.250798
#Ankara,57,313.0,58158.0,185.808307,7194.0,22.984026,409.0,1.306709,2350.814230,7.510589,12.420,271.031,29.554,60.1001,0.039681,0.865914,0.094422,0.192013
#Athens,99,1000.0,186752.0,186.752000,24268.0,24.268000,1503.0,1.503000,7170.591520,7.170592,42.164,865.646,92.190,176.3919,0.042164,0.865646,0.092190,0.176392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
#Zurich,3,485.0,94012.0,193.839175,11973.0,24.686598,731.0,1.507216,3528.782923,7.275841,11.206,431.700,42.100,100.8435,0.023105,0.890103,0.086804,0.207925
#Taipei,8,17337.0,2505360.0,144.509431,321964.0,18.570918,25826.0,1.489646,125070.218467,7.214063,375.559,14812.023,2149.449,4884.2746,0.021662,0.854359,0.123980,0.281725
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#telaviv,50,17612.0,2468640.0,140.168067,325193.0,18.464286,25500.0,1.447876,122706.647131,6.967218,926.040,14559.546,2126.443,3204.5068,0.052580,0.826683,0.120738,0.181950


In [36]:
sc100_features = sc100_features.drop(columns=['total_tweet_len','total_num_words','total_num_sentences','total_avg_word_len','neg','neu','pos','compound'])

In [37]:
# create a new column with lists of merged tweets grouped by city
sc100_features = pd.concat([sc100_features, smartcities100.groupby('query')['content'].apply(lambda x: x.tolist())], axis=1)

In [38]:
# convert the lists into strings
sc100_features['content'] = sc100_features['content'].apply(lambda x: str(x).strip('[]'))

In [39]:
# count number of words for each city
sc100_features['num_words'] = sc100_features['content'].apply(lambda x: len(x))

In [40]:
# lower characters inside column
sc100_features['content'] = sc100_features['content'].apply(lambda x: x.lower())

In [41]:
# remove # inside column
import re
sc100_features['content'] = sc100_features['content'].apply(lambda x: re.sub(r'#',' ', x))

In [42]:
# Remove NaN row and reset index
sc100_features = sc100_features.reset_index().dropna()

In [44]:
# clean column names
sc100_features.columns = ['city_id', 'rank2020', 'num_tweets','avg_tweet_len', 'avg_num_words', 'avg_num_sentences', 'avg_word_len', 'avg_neg', 'avg_neu', 'avg_pos', 'avg_compound', 'tweets_merged', 'total_num_words']
sc100_features

,city_id,rank2020,num_tweets,avg_tweet_len,avg_num_words,avg_num_sentences,avg_word_len,avg_neg,avg_neu,avg_pos,avg_compound,tweets_merged,total_num_words
0,#AbuDhabi,42,832.0,193.930288,25.002404,1.435096,7.076026,0.021933,0.860132,0.117936,0.299883,'a delegation from @aau_ae visited @burjeelme...,166602
1,#Abuja,107,828.0,209.646135,27.042271,1.437198,7.075491,0.018193,0.856400,0.125396,0.356573,"'brother abuja https://t.co/6j3yoip8jm', 'our...",180295
2,#Amsterdam,9,1000.0,180.311000,22.627000,1.520000,7.579219,0.024567,0.875009,0.100439,0.250798,'herfst in amsterdam 🍂🍃🍁 https://t.co/f0vkrmq...,185912
3,#Ankara,57,313.0,185.808307,22.984026,1.306709,7.510589,0.039681,0.865914,0.094422,0.192013,'i had a great and very joyful flight today ✈️...,60230
4,#Athens,99,1000.0,186.752000,24.268000,1.503000,7.170592,0.042164,0.865646,0.092190,0.176392,'georgia ga garunoffs\n sparta athens ciara...,192839
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,#Zhuhai,62,422.0,189.907583,25.023697,1.637441,7.223551,0.029555,0.904628,0.065820,0.117996,'3 injured in an explosion of a hotel in zhuh...,82619
105,#Zurich,3,485.0,193.839175,24.686598,1.507216,7.275841,0.023105,0.890103,0.086804,0.207925,'the sky is clearing up \n\n zurich oberland ...,96701
106,#Taipei,8,17337.0,144.509431,18.570918,1.489646,7.214063,0.021662,0.854359,0.123980,0.281725,"' taiwan is full of unique, stunning landscape...",2589903
108,#telaviv,50,17612.0,140.168067,18.464286,1.447876,6.967218,0.052580,0.826683,0.120738,0.181950,'@simonsoesan @emgoyaars helemaal mee eens ! h...,2550356


In [45]:
# clean city_id by removing # using regex
sc100_features['city_id'] = sc100_features['city_id'].apply(lambda x: re.sub(r'#','', str(x)))

In [46]:
# lower characters
sc100_features['city_id'] = sc100_features['city_id'].str.lower()

In [47]:
sc100_features['num_tweets'] = sc100_features['num_tweets'].apply(np.int64)
sc100_features

,city_id,rank2020,num_tweets,avg_tweet_len,avg_num_words,avg_num_sentences,avg_word_len,avg_neg,avg_neu,avg_pos,avg_compound,tweets_merged,total_num_words
0,abudhabi,42,832,193.930288,25.002404,1.435096,7.076026,0.021933,0.860132,0.117936,0.299883,'a delegation from @aau_ae visited @burjeelme...,166602
1,abuja,107,828,209.646135,27.042271,1.437198,7.075491,0.018193,0.856400,0.125396,0.356573,"'brother abuja https://t.co/6j3yoip8jm', 'our...",180295
2,amsterdam,9,1000,180.311000,22.627000,1.520000,7.579219,0.024567,0.875009,0.100439,0.250798,'herfst in amsterdam 🍂🍃🍁 https://t.co/f0vkrmq...,185912
3,ankara,57,313,185.808307,22.984026,1.306709,7.510589,0.039681,0.865914,0.094422,0.192013,'i had a great and very joyful flight today ✈️...,60230
4,athens,99,1000,186.752000,24.268000,1.503000,7.170592,0.042164,0.865646,0.092190,0.176392,'georgia ga garunoffs\n sparta athens ciara...,192839
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,zhuhai,62,422,189.907583,25.023697,1.637441,7.223551,0.029555,0.904628,0.065820,0.117996,'3 injured in an explosion of a hotel in zhuh...,82619
105,zurich,3,485,193.839175,24.686598,1.507216,7.275841,0.023105,0.890103,0.086804,0.207925,'the sky is clearing up \n\n zurich oberland ...,96701
106,taipei,8,17337,144.509431,18.570918,1.489646,7.214063,0.021662,0.854359,0.123980,0.281725,"' taiwan is full of unique, stunning landscape...",2589903
108,telaviv,50,17612,140.168067,18.464286,1.447876,6.967218,0.052580,0.826683,0.120738,0.181950,'@simonsoesan @emgoyaars helemaal mee eens ! h...,2550356


# extract weigh of BoW for each city_id

In [48]:
# define data transformations
stemmer = PorterStemmer()

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    tokens = word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

vect = CountVectorizer(tokenizer=tokenize, stop_words='english')

# define each BoW as a list of thematic words (Lexicons)
smartcity = ['digitaltransformation','2.0','smartcity','iot','smart','city','ai','smartcities','innovation','cities','5g','data','bigdata','technology','future','tech','digital','digitaltransformation','machinelearning','analytics','solutions','cloud','datascience','network','security','cybersecurity','urban','people','artificialintelligence','smarthome','internetofthings','autonomous','smart','technologies','robotics','software','research','monitoring','street','sensors','futureofwork','planning','governance','transportation','safecity','building','robot','industry40','startups','development','smartthings','urbanplanning','connected','internet','sustainable','waste','emergingtech','programming','environment','safety','lighting','safety','roads','workplaces','smartbuildings','driverless','selfdriving','smartmobility','futureworkspace','digitalcity','deeplearning','datadriven','application','robots','publicsafety','energyefficiency','labgov','citizens','transformation','controlroom','publicspace','airquality','efficiency','era','smarter','urbaninnovation','internetofeverything','newtown','iotforall','construction','privacy','surveillance','drones','smartbuilding','smartgrid','smartgrids','cybersecurity','policy','powergrid','smartcities','iot','electricity','smartmetering','smartmeters','technology','microgrids','microgrid','smartcity','powergrids','iot','ai','machinelearning','bigdata','serverless','cybersecurity','devcommunity','programming','coding','cloudcomputing','codenewbie','100daysofmlcode','womenintech','innovation','deeplearning','security','machine','digital','industrial','artificial','computing','robot','wearables','artificial','networks','drones','connectivity','wireless','urbanplanning','city','urban','cities','smartcities','architecture','urbanism','urbandesign','cityplanning','urbanization','futurecities','urbaninnovation','bigdata','cityplanner','architects','citylab','urbanplanner','urbanplanners','cityplanners','urbandata','townplanning','planners','neighborhoods','sustainablecities','tacticalurbanism','urbanmobility','pedestrians','neighborhood','citiesforpeople','neighbourhood','rethinkreplace','constructionmanagement','suburban','futurecity','cityvisionnaire','smarttechnology','citygoals','civilengineering','digitaltwin','bigdata','digitaltransformation','pedestrian-friendly','surveillance','technologies','urbandevelopment','cities','urban','urbanplanning'] 
civictech = ['aiethics','empowerment','learned','communityengagement','communitydevelopment','nonprofit','notforprofit','collaboration','vulnerable','diversity','egov','egovernance','inclusion','ethics','social','engagement','crowdsourcing','humanrights','citizenship','civic','opendata','referendum','rights','citizens','open','social','people','sharingeconomy','hack','citizens','communities','civictech','inequalities','equity','participatory','social','youthempowerment','govtech','localgov','civic','stateandlocal','community','governmentit','statelocalit','opendata','sharing','including','volunteer','social','people','shared','learn','opengov','localgovernment','education','hackathon','civictechto','techforgood','discussion','communities','local','digitalgov','future','online','governments','webmapping','capacity','accessibility','discuss','ethics','citizens','together','initiative','transparency','data4good','publicinteresttech','democracy','volunteers','society','makers','ecitizenship','eresidency','citizenship','egovernment','esafety','digitaltransformation','policy','social','technology','future','citizen','ecivility','digitaleconomy','ideagov','egov','people','edemocracy','eparticipation','education','citizens','progressive','security','ecitizenship','inclusive','watch','startups','concept','digitalnation','netgov','app','privacy','egovernance','tool','democratic','eublockchain','digitaleu','digitalnomad','apps','residents','populationeu','migrationpolicy','migrationsystem','migration','voted','govtech','smartcities','humandevelopment','erisks','ecitizen','artificial','intelligence','opendata','empowerment','leadership','growth','success','youth','wisdom','education','relationship','powerinpurpose','community','beyondlimits','powerwithin','support','empower','empowering','equality','enlightment','selfdiscovery','confidence','selfempowerment','leaders','forward','diversity','empowered','successmindset','cybersecurity','government','security','technology','opendata','open','openscience','opensource','opendataday','community','citizenscience','citizen','citizenship','openaccess','transparence','openbanking','womenwhocode','openapis','policy','grants','decisions','institutions','opengov','social','busopendata','datadriven','citizen','openstreetmap','researchdata','engagement','citizens']
infrastructure = ['supply','platform','blockchain','system','internet','battery','batterystorage','energystorage','mobility','storage','blockchain','serverless','grids','blockchain','standards','realestate','construction','driverless','selfdriving','smartmobility','transportation','roads', 'building','traffic','autonomousvehicles','streets','platform','transportation','road','publicspace','realestate','monuments','housing','selfdrivingcars','high-speed','real-estate','mobility','highways','transportation','transport','automotive','transit','buses','cars','futureofmobility','car','travel','publictransport','autonomous','bus','auto','shared','traffic','delivery','emissions','5g','services','connectivity','van','trucks','vehicles','commute','electricvehicles','smartmobility','vehicle','logistics','multimodal','connected','metropolitan','efficiency','monitoring','providers','mobile','futurride','autonomousvehicle','engineer','seat','robotics','parking','cycling','driver','flying','rail','moving','wheelchair','automobile','walks','electricvehicle','electriccars','platform','cctv','drivers','driverless','systems','roads','drones','aviation','selfdriving','robots','electriccar','device','bike','autos','urbanmobility','speed','selfdrivingcars','network','street','urbanplanning','autocar','drive','electrification','streets','driverlesscars','bicycle','zero-emission','engineering','artificialintelligence','emobility','fleet','scooters','bikes','journey','manufacturers','route','logistics','driver','trucking','delivery','shipping','traffic','transport','freight','rentals','road','drivers','bus','mobility','rail','cargo','cars','trucks','trucker','fleet','mechanic','infrastructure','railways','diesel','taxi','truckdriver','truckers','shipments','driving','flights','pickup','airfreight','intelligentlogistics','bicycle','cycling','port','wheels','motors','machinery','ferry','infrastructure','bridge','network','architecture','building','construction','transport','metro','roads','highways','railways','realestate','housing','train','civilengineering','airport','station','connectivity','stations','tunnel','airports','equipment','bridges','km','electrical','highway','tunnels','road']
sustainability = ['resource','resources','sustainablefinance','cyberresilience','recycling','resilient','batteries','emission','resilience','energytransition','sustainability','biodiversity','sustainabledevelopment','resources','sustainableurbanplanning','sustainabletransport','sustainable','environment','climatechange','energy','climate','innovation','circulareconomy','environmental','ecofriendly','nature','planet','carbon','recycling','climateaction','plastic','renewableenergy','zerowaste','water','waste','solar','agriculture','recycle','savetheplanet','pollution','emissions','sustainablefashion','hydrogen','eco','sustainableliving','reuse','clean','recycled','greenliving','renewable','circular','gogreen','biodiversity','sustainable','renewables','climatecrisis','organic','eco-friendly','earth','sustainabledevelopment','electric','electricity','efficiency','electricvehicles','recyclable','footprint','sustainablefinance','ocean','greener','co2','plasticfree','climateemergency','greenenergy','waterconservation','carbonfootprint','greenbuilding','energytransition','sustainablity','consumption','globalwarming','resilience','reusable','battery','decompose','aquaculture','oil','compost','circularity','plastics','environment','ecoconscious','savetheplanet','green_action','eco-outreach','eco-friendly','environmental_activism','climateactivism','energyefficiency','agtech','reduction','lessismore','responsibility','🌍','cleanup','cleantech','ethical','sustainably','ecofashion','foodtech','emobility','ecosystem','passivehouse','farmtotable','nature','wildlife','planet','ecofriendly','trees','conservation','climatecrisis','savetheplanet','clean','recycle','natural','biodiversity','tree','plants','emissions','reduce','ecology','zerowaste','forests','globalwarming','garden','species','gogreen','farming','agriculture','eco','plasticpollution','soil','greener','plasticfree','farmers','extinction','reuse','recycled','renewableenergy','circulareconomy','environmentally','deforestation','oceans','fuel','naturelovers','microplastics','airpollution','rivers','plastics','renewables','climateemergency','gas','sustainablefashion','environmentaljustice','eco-friendly','bees','fossil','wastewater','fornature','wildfires','greenhouse','ecosystems','renewables','oilandgas','hydrogen','cleanenergy','greenspaces','resilience','climateemergency','sustainablemobility','sustainabledevelopment','naturebasedsolutions','energyefficiency','actonclimate','solar','renewableenergy','alternativeenergy','solarpower','sustainable','environment','solarpanels','solarpv','scienceandenvironment','solarsystems','solarenergy','energymanagement','greennewdeal','solarenergy']
governance = ['partnership','strategies','leadership','responsibility','regulations','ceo','policy','democracy2.0','elections','voter','collaboration','gov','governance','planning','management','localgov','project','partnership','partners','partner','leader','leaders','politicalparties','administration','decentralized','democracy2.0','political','society','minister','vision','crossbordercooperation','political','politics','democratic','government','politicians','elections','voter','public','commonwealth','politicalprogress','voting','debate','national','votingawareness','president','mayor','governance','parliament','campaign','liquiddemocracy','voters','brigade','constitutional','law','country','legalhack','decentralization','representatives','communityactivity','republican','policing','policy','policies','administration','governance','leadership','management','security','cybersecurity','ceo','corpgov','government','public','strategy','corruption','democracy','boardofdirectors','cio','policy','technology','leaders','boards','role','members','report','planning','throwback','organizations','national','directorship','structures','trust','directors','decentralized','countries','chair','discussion','conference','transparency','compensationcommittee','auditcommittee','cto','visionarytalks','riskmanagement','organisations','office','legal','regulations','audit','insights','corporategovernance','organization','voting','governing','organisation','policies','project','executive','responsible','leader','survey','interview','approach','staff','benefits','recruitment','regulatory','reporting','severity','obstacle','presidential','country','elections','institutions','alternative','digitaltransformation','freedom','nation','datavault','datawarehouse','regulation','processes','foundation','critical','control','candidate','promised','inauguration','anticorruption','executives','publicpolicy','principles','strategic','sharing','governments','federated','elected','framework','schedule','risks','certificate','meritocracy','stakeholders','partnership','society','equality','decisions','structure','implement','commitment','decision','driven','citiesalliance','assessment','corporations','cryptocurrencies','dataprotection']
entrepreneurship = ['freemarket','investors','crypto','financial','accountability','enterprise','assetmanagement','accountable','company','finance','corporate','contractors','workforce','digitalbanking','entrepreneur','newmarket','investment', 'market','coding','startup','development','maker','fintech','economy','business','economic','finance','markets','investing','stocks','investment','gdp','money','financial','economics','trade','crypto','businesses','investors','retail','bitcoin','employment','job','startups','growth','work','entrepreneur','unemployment','entrepreneurship','sales','manufacturing','working','corporate','companies','funds','jobless','cryptocurrency','economicrecovery','smallbusiness','workers','investments','traders','budget','assets','growing','industries','asset','invest','startup','funding','development','currency','banks','monetary','debt','skills','fintech','fiscal','entrepreneurs','deficit','banking','benefits','rates','paying','billing','billingsoftware','earnings','workfromhome','accounting','personalfinance','ratings','accountant','treasury','equities','subsidies','opportunities','profits','stockstowatch','branding','marketing','customer','firms','localbusiness','capitalism','consumers','startup','startups','innovation','mindset','smallbusiness','businessowner','entrepreneurlife','digitalmarketing','management','founder','branding','growth','company','founders','funding','product','businesses','businessgrowth','sales','coaching','investment','entrepreneurial','venture','businessangel','capital','smallbusinessowner','businessman','customers','businesscoach','speaker','finance','market','skills','coding','ecommerce','entrepreneurmindset','startuplife','pitch','onlinebusiness','employment','businesswoman','socialmediamarketing','girlswhocode','crypto','womeninbusiness','investors','selfemployment','accelerator','incubator','designthinking','growthhacking','enterprise','changemakers','startupbusiness','businessideas','mentors','businesscoaching','businessmotivation','businessgoals','ventures','skill','innovators','prototype','growthmindset','pitching','innovations','businesslife','businessstrategy','smallbusinessowners','financial','companies','entrepreneurlifestyle','franchise','manufactures','beyourownboss']

In [49]:
sma_vec = vect.fit(smartcity)
sc100_features['sma_bow'] = sc100_features['tweets_merged'].apply(lambda x: np.sum(sma_vec.transform([x]).toarray()))

In [50]:
civ_vec = vect.fit(civictech)
sc100_features['civ_bow'] = sc100_features['tweets_merged'].apply(lambda x: np.sum(civ_vec.transform([x]).toarray()))

In [51]:
inf_vec = vect.fit(infrastructure)
sc100_features['inf_bow'] = sc100_features['tweets_merged'].apply(lambda x: np.sum(inf_vec.transform([x]).toarray()))

In [52]:
sus_vec = vect.fit(sustainability)
sc100_features['sus_bow'] = sc100_features['tweets_merged'].apply(lambda x: np.sum(sus_vec.transform([x]).toarray()))

In [53]:
gov_vec = vect.fit(governance)
sc100_features['gov_bow'] = sc100_features['tweets_merged'].apply(lambda x: np.sum(gov_vec.transform([x]).toarray()))

In [54]:
ent_vec = vect.fit(entrepreneurship)
sc100_features['ent_bow'] = sc100_features['tweets_merged'].apply(lambda x: np.sum(ent_vec.transform([x]).toarray()))

In [55]:
sc100_features

,city_id,rank2020,num_tweets,avg_tweet_len,avg_num_words,avg_num_sentences,avg_word_len,avg_neg,avg_neu,avg_pos,avg_compound,tweets_merged,total_num_words,sma_bow,civ_bow,inf_bow,sus_bow,gov_bow,ent_bow
0,abudhabi,42,832,193.930288,25.002404,1.435096,7.076026,0.021933,0.860132,0.117936,0.299883,'a delegation from @aau_ae visited @burjeelme...,166602,313,290,345,258,312,328
1,abuja,107,828,209.646135,27.042271,1.437198,7.075491,0.018193,0.856400,0.125396,0.356573,"'brother abuja https://t.co/6j3yoip8jm', 'our...",180295,249,200,320,66,287,394
2,amsterdam,9,1000,180.311000,22.627000,1.520000,7.579219,0.024567,0.875009,0.100439,0.250798,'herfst in amsterdam 🍂🍃🍁 https://t.co/f0vkrmq...,185912,381,241,339,103,195,299
3,ankara,57,313,185.808307,22.984026,1.306709,7.510589,0.039681,0.865914,0.094422,0.192013,'i had a great and very joyful flight today ✈️...,60230,49,74,68,21,108,106
4,athens,99,1000,186.752000,24.268000,1.503000,7.170592,0.042164,0.865646,0.092190,0.176392,'georgia ga garunoffs\n sparta athens ciara...,192839,247,324,284,104,272,315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,zhuhai,62,422,189.907583,25.023697,1.637441,7.223551,0.029555,0.904628,0.065820,0.117996,'3 injured in an explosion of a hotel in zhuh...,82619,182,106,228,62,122,104
105,zurich,3,485,193.839175,24.686598,1.507216,7.275841,0.023105,0.890103,0.086804,0.207925,'the sky is clearing up \n\n zurich oberland ...,96701,115,111,158,39,84,142
106,taipei,8,17337,144.509431,18.570918,1.489646,7.214063,0.021662,0.854359,0.123980,0.281725,"' taiwan is full of unique, stunning landscape...",2589903,4526,3754,5565,1068,3713,2838
108,telaviv,50,17612,140.168067,18.464286,1.447876,6.967218,0.052580,0.826683,0.120738,0.181950,'@simonsoesan @emgoyaars helemaal mee eens ! h...,2550356,5995,4892,3750,1184,4071,4078


In [56]:
# Save new .csv file with lastly created columns
sc100_features.to_csv('sc100_featureBoW.csv', index=False)